# Dynamic Policy Impacts Analysis

This notebook calculates both static and dynamic budgetary impacts for all reform options.
Dynamic scoring incorporates CBO labor supply response elasticities to estimate behavioral effects.

**Note**: This notebook generates the data files used in the revenue impacts analysis. Run this first to update the static and dynamic impact estimates.

In [ ]:
# Import necessary libraries
import sys
sys.path.append('../src')
import os

# Ensure we're in the repo root for data paths
if os.path.basename(os.getcwd()) == 'analysis':
    os.chdir('..')

import pandas as pd
import numpy as np
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform
from reforms import REFORMS
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

## Setup: CBO Labor Response Parameters

Define the CBO labor supply elasticities used for dynamic scoring:

In [2]:
# CBO labor supply elasticities
# Updated for PR #6681: Uses new parameter structure with automatic age multipliers
# Base elasticities apply to under-65; 65+ automatically get 2x multiplier (default)

CBO_LABOR_PARAMS = {
    # Income elasticity (base for under 65, auto 2x for 65+)
    "gov.simulation.labor_supply_responses.elasticities.income.base": {
        "2024-01-01.2100-12-31": -0.05
    },
    # Age multiplier for income elasticity (default 2.0, can override)
    "gov.simulation.labor_supply_responses.elasticities.income.age_multiplier_over_threshold": {
        "2024-01-01.2100-12-31": 2.0
    },
    
    # Substitution elasticities by income decile for primary earners (base, auto 2x for 65+)
    "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.1": {
        "2024-01-01.2100-12-31": 0.31
    },
    "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.2": {
        "2024-01-01.2100-12-31": 0.28
    },
    "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.3": {
        "2024-01-01.2100-12-31": 0.27
    },
    "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.4": {
        "2024-01-01.2100-12-31": 0.27
    },
    "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.5": {
        "2024-01-01.2100-12-31": 0.25
    },
    "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.6": {
        "2024-01-01.2100-12-31": 0.25
    },
    "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.7": {
        "2024-01-01.2100-12-31": 0.22
    },
    "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.8": {
        "2024-01-01.2100-12-31": 0.22
    },
    "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.9": {
        "2024-01-01.2100-12-31": 0.22
    },
    "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.10": {
        "2024-01-01.2100-12-31": 0.22
    },
    # Substitution elasticity for secondary earners (all deciles)
    "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.secondary": {
        "2024-01-01.2100-12-31": 0.27
    },
    # Age multiplier for substitution elasticity (default 2.0, can override)
    "gov.simulation.labor_supply_responses.elasticities.substitution.age_multiplier_over_threshold": {
        "2024-01-01.2100-12-31": 2.0
    }
}

print("CBO labor supply elasticities configured (PR #6681 STRUCTURE)")
print("=" * 70)
print(f"Base income elasticity (under 65): {CBO_LABOR_PARAMS['gov.simulation.labor_supply_responses.elasticities.income.base']['2024-01-01.2100-12-31']}")
print(f"Income age multiplier (65+): {CBO_LABOR_PARAMS['gov.simulation.labor_supply_responses.elasticities.income.age_multiplier_over_threshold']['2024-01-01.2100-12-31']}x")
print(f"→ Effective income elasticity for 65+: {CBO_LABOR_PARAMS['gov.simulation.labor_supply_responses.elasticities.income.base']['2024-01-01.2100-12-31'] * CBO_LABOR_PARAMS['gov.simulation.labor_supply_responses.elasticities.income.age_multiplier_over_threshold']['2024-01-01.2100-12-31']}")
print()
print(f"Base substitution elasticity (primary, decile 1, under 65): {CBO_LABOR_PARAMS['gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.1']['2024-01-01.2100-12-31']}")
print(f"Substitution age multiplier (65+): {CBO_LABOR_PARAMS['gov.simulation.labor_supply_responses.elasticities.substitution.age_multiplier_over_threshold']['2024-01-01.2100-12-31']}x")
print(f"→ Effective substitution elasticity for 65+ (decile 1): {CBO_LABOR_PARAMS['gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.1']['2024-01-01.2100-12-31'] * CBO_LABOR_PARAMS['gov.simulation.labor_supply_responses.elasticities.substitution.age_multiplier_over_threshold']['2024-01-01.2100-12-31']}")
print()
print("Note: Age multiplier of 2.0 is conservative estimate from French (2005)")
print("      Range in literature: 1.5-3.25x for 65+ vs under-65 populations")

CBO labor supply elasticities configured (PR #6681 STRUCTURE)
Base income elasticity (under 65): -0.05
Income age multiplier (65+): 2.0x
→ Effective income elasticity for 65+: -0.1

Base substitution elasticity (primary, decile 1, under 65): 0.31
Substitution age multiplier (65+): 2.0x
→ Effective substitution elasticity for 65+ (decile 1): 0.62

Note: Age multiplier of 2.0 is conservative estimate from French (2005)
      Range in literature: 1.5-3.25x for 65+ vs under-65 populations


## Helper Functions

In [3]:
def create_dynamic_reform(base_reform_dict, labor_params=CBO_LABOR_PARAMS):
    """
    Combine a base reform with CBO labor supply elasticities.
    
    Args:
        base_reform_dict: Dictionary of reform parameters  
        labor_params: Dictionary of labor supply elasticity parameters
    
    Returns:
        Reform object with combined parameters
    """
    # Combine dictionaries (both use DOT notation for date ranges)
    combined_dict = {**base_reform_dict, **labor_params}
    
    return Reform.from_dict(combined_dict, country_id="us")


def get_reform_dict(reform_func):
    """
    Extract the underlying parameter dictionary from a reform function.
    
    This works by importing the helper functions from reforms.py that return
    the raw dictionaries before they're wrapped in Reform objects.
    """
    # Import the helper functions that return dicts
    from reforms import (
        eliminate_ss_taxation, tax_85_percent_ss, tax_100_percent_ss,
        extend_senior_deduction, add_ss_tax_credit, eliminate_senior_deduction,
        enable_employer_payroll_tax
    )
    
    # Map each reform function to its underlying dict-returning function(s)
    # This is based on the structure in reforms.py
    reform_func_name = reform_func.__name__
    
    if reform_func_name == "get_option1_reform":
        return eliminate_ss_taxation()
    elif reform_func_name == "get_option2_reform":
        return tax_85_percent_ss()
    elif reform_func_name == "get_option3_reform":
        return {**tax_85_percent_ss(), **extend_senior_deduction()}
    elif reform_func_name == "get_option4_reform":
        return {**tax_85_percent_ss(), **add_ss_tax_credit(500), **eliminate_senior_deduction()}
    elif reform_func_name == "get_option5_reform":
        return {**eliminate_ss_taxation(), **enable_employer_payroll_tax(1.0)}
    elif reform_func_name == "get_option6_reform":
        # Option 6: Phased Roth-Style Swap with proper leaf parameters
        reform_dict = {
            "gov.contrib.crfb.tax_employer_payroll_tax.in_effect": {
                "2026-01-01.2100-12-31": True
            },
            "gov.contrib.crfb.tax_employer_payroll_tax.percentage": {
                "2026": 0.1307,  # 1/7.65
                "2027": 0.2614,  # 2/7.65
                "2028": 0.3922,  # 3/7.65
                "2029": 0.5229,  # 4/7.65
                "2030": 0.6536,  # 5/7.65
                "2031": 0.7843,  # 6/7.65
                "2032": 0.9150,  # 7/7.65
                "2033-01-01.2100-12-31": 1.0  # Full from 2033 onward
            }
        }
        
        # Phase down base rate parameters (MUST use leaf parameters)
        base_years = [2029, 2030, 2031, 2032, 2033, 2034, 2035, 2036, 2037]
        base_values = [0.45, 0.40, 0.35, 0.30, 0.25, 0.20, 0.15, 0.10, 0.05]
        
        for param_name in ["benefit_cap", "excess"]:
            param_path = f"gov.irs.social_security.taxability.rate.base.{param_name}"
            reform_dict[param_path] = {}
            for year, value in zip(base_years, base_values):
                reform_dict[param_path][str(year)] = value
            reform_dict[param_path]["2038-01-01.2100-12-31"] = 0
        
        # Phase down additional rate parameters (MUST use leaf parameters)
        add_years = list(range(2029, 2045))
        add_values = [0.80, 0.75, 0.70, 0.65, 0.60, 0.55, 0.50, 0.45, 0.40,
                      0.35, 0.30, 0.25, 0.20, 0.15, 0.10, 0.05]
        
        for param_name in ["benefit_cap", "bracket", "excess"]:
            param_path = f"gov.irs.social_security.taxability.rate.additional.{param_name}"
            reform_dict[param_path] = {}
            for year, value in zip(add_years, add_values):
                reform_dict[param_path][str(year)] = value
            reform_dict[param_path]["2045-01-01.2100-12-31"] = 0
        
        return reform_dict
        
    elif reform_func_name == "get_option7_reform":
        return eliminate_senior_deduction()
    elif reform_func_name == "get_option8_reform":
        return tax_100_percent_ss()
    else:
        raise ValueError(f"Unknown reform function: {reform_func_name}")


def calculate_revenue_impact(reform, year, baseline_income_tax):
    """
    Calculate revenue impact for a given reform and year.
    
    Args:
        reform: Reform object (can be static or dynamic)
        year: Year to calculate impact for
        baseline_income_tax: Pre-computed baseline income tax array
    
    Returns:
        Revenue impact in dollars (positive = revenue gain, negative = revenue loss)
    """
    # Create reformed simulation
    reform_sim = Microsimulation(reform=reform)
    
    # Calculate reformed income tax
    reform_income_tax = reform_sim.calculate("income_tax", map_to="household", period=year)
    
    # JCT convention: reformed - baseline (positive = more revenue)
    revenue_impact = reform_income_tax.sum() - baseline_income_tax.sum()
    
    return revenue_impact


def compute_baselines(years):
    """Pre-compute baselines for all years to avoid redundant calculations.
    
    Args:
        years: List of years to compute baselines for
    
    Returns:
        Dictionary mapping years to baseline income tax arrays
    """
    print("Pre-computing baselines for all years...")
    baselines = {}
    
    for year in years:
        print(f"  Computing baseline for {year}...")
        baseline = Microsimulation()
        baseline_income_tax = baseline.calculate("income_tax", map_to="household", period=year)
        baselines[year] = baseline_income_tax
    
    print("Baseline computation complete!\n")
    return baselines


print("Helper functions defined")

Helper functions defined


## Compute Baselines

Pre-compute baseline simulations for all years. This cell must be run before either static or dynamic calculations.

In [4]:
# Years to analyze (used by both static and dynamic calculations)
YEARS = list(range(2026, 2036))

# Pre-compute baselines for all years
baselines = compute_baselines(YEARS)

print(f"✓ Baselines computed for {len(YEARS)} years: {YEARS[0]}-{YEARS[-1]}")
print(f"  These baselines can now be used for both static and dynamic calculations.")

Pre-computing baselines for all years...
  Computing baseline for 2026...
  Computing baseline for 2027...
  Computing baseline for 2028...
  Computing baseline for 2029...
  Computing baseline for 2030...
  Computing baseline for 2031...
  Computing baseline for 2032...
  Computing baseline for 2033...
  Computing baseline for 2034...
  Computing baseline for 2035...
Baseline computation complete!

✓ Baselines computed for 10 years: 2026-2035
  These baselines can now be used for both static and dynamic calculations.


## Static Impact Calculation

Calculate static budgetary impacts (without behavioral responses) for all reforms across 2026-2035.

**Note:** You must run the "Compute Baselines" cell above before running this cell.

In [ ]:
# Storage for results
static_results = []

# Create data directory if it doesn't exist
import os
os.makedirs('data', exist_ok=True)

# Load existing checkpoint if it exists to avoid duplicates
checkpoint_file = 'data/policy_impacts_static_checkpoint.csv'
if os.path.exists(checkpoint_file):
    existing_df = pd.read_csv(checkpoint_file)
    static_results = existing_df.to_dict('records')
    print(f"📂 Loaded {len(static_results)} existing static results from checkpoint")
    print(f"   Reforms already computed: {sorted(existing_df['reform_id'].unique())}")
    print(f"   Years covered: {sorted(existing_df['year'].unique())}")
    print()

print("\n" + "="*80)
print("STATIC IMPACT CALCULATIONS")
print("="*80)
print(f"Analyzing {len(REFORMS)} reforms across {len(YEARS)} years = {len(REFORMS) * len(YEARS)} calculations\n")

for reform_id, reform_config in tqdm(REFORMS.items(), desc="Reforms"):
    reform_name = reform_config['name']
    reform_func = reform_config['func']
    
    print(f"\nProcessing {reform_id}: {reform_name}")
    
    try:
        # Get the static reform (already works with the existing Reform objects from reforms.py)
        static_reform = reform_func()
        
        for year in tqdm(YEARS, desc=f"  Years ({reform_id})", leave=False):
            print(f"  Calculating {year}...", end=' ')
            
            impact = calculate_revenue_impact(static_reform, year, baselines[year])
            
            static_results.append({
                'reform_id': reform_id,
                'reform_name': reform_name,
                'year': year,
                'revenue_impact': impact,
                'scoring_type': 'static'
            })
            
            print(f"${impact/1e9:.2f}B")
        
        # Deduplicate before saving checkpoint (keep most recent calculation)
        checkpoint_df = pd.DataFrame(static_results)
        checkpoint_df = checkpoint_df.drop_duplicates(
            subset=['reform_id', 'year', 'scoring_type'], 
            keep='last'
        )
        checkpoint_df.to_csv('data/policy_impacts_static_checkpoint.csv', index=False)
        print(f"  ✓ Checkpoint saved ({len(checkpoint_df)} unique records)")
        
    except Exception as e:
        print(f"  ✗ ERROR processing {reform_id}: {type(e).__name__}: {e}")
        print(f"  Continuing with next reform...")
        import traceback
        traceback.print_exc()

# Convert to DataFrame and deduplicate final results
static_df = pd.DataFrame(static_results)
if len(static_df) > 0:
    static_df = static_df.drop_duplicates(
        subset=['reform_id', 'year', 'scoring_type'], 
        keep='last'
    )

print("\n" + "="*80)
print("STATIC IMPACTS SUMMARY")
print("="*80)

if len(static_df) > 0:
    # Show 10-year totals
    static_totals = static_df.groupby(['reform_id', 'reform_name'])['revenue_impact'].sum() / 1e9
    print("\n10-Year Static Impacts (2026-2035, Billions):")
    for reform_id, total in static_totals.items():
        print(f"  {reform_id[0]}: ${total:,.1f}B")
    
    print(f"\nTotal records: {len(static_df)}")
else:
    print("⚠ No static results calculated")

## Dynamic Impact Calculation

Calculate dynamic budgetary impacts incorporating CBO labor supply elasticities for 2026 only.
This captures behavioral responses to tax changes.

**Note:** You must run the "Compute Baselines" cell above before running this cell.

In [ ]:
# Storage for dynamic results
dynamic_results = []

# Create data directory if it doesn't exist
import os
os.makedirs('data', exist_ok=True)

# Load existing checkpoint if it exists to avoid duplicates
checkpoint_file = 'data/policy_impacts_dynamic_checkpoint.csv'
if os.path.exists(checkpoint_file):
    existing_df = pd.read_csv(checkpoint_file)
    dynamic_results = existing_df.to_dict('records')
    print(f"📂 Loaded {len(dynamic_results)} existing dynamic results from checkpoint")
    print(f"   Reforms already computed: {sorted(existing_df['reform_id'].unique())}")
    print(f"   Years covered: {sorted(existing_df['year'].unique())}")
    print()

# Only calculate dynamic for 2026 (single year)
DYNAMIC_YEARS = [2026]

print("\n" + "="*80)
print("DYNAMIC IMPACT CALCULATIONS")
print("="*80)
print(f"Analyzing {len(REFORMS)} reforms for year {DYNAMIC_YEARS[0]} only = {len(REFORMS)} calculations\n")
print("Note: Dynamic scoring with CBO labor elasticities is computationally expensive.")
print("      Calculating only 2026 to demonstrate behavioral effects.\n")

for reform_id, reform_config in tqdm(REFORMS.items(), desc="Reforms"):
    reform_name = reform_config['name']
    reform_func = reform_config['func']
    
    print(f"\nProcessing {reform_id}: {reform_name}")
    
    try:
        # Get the reform dict and combine with labor elasticities
        reform_dict = get_reform_dict(reform_func)
        dynamic_reform = create_dynamic_reform(reform_dict, CBO_LABOR_PARAMS)
        
        for year in DYNAMIC_YEARS:
            print(f"  Calculating {year}...", end=' ')
            
            impact = calculate_revenue_impact(dynamic_reform, year, baselines[year])
            
            dynamic_results.append({
                'reform_id': reform_id,
                'reform_name': reform_name,
                'year': year,
                'revenue_impact': impact,
                'scoring_type': 'dynamic'
            })
            
            print(f"${impact/1e9:.2f}B")
        
        # Deduplicate before saving checkpoint (keep most recent calculation)
        checkpoint_df = pd.DataFrame(dynamic_results)
        checkpoint_df = checkpoint_df.drop_duplicates(
            subset=['reform_id', 'year', 'scoring_type'], 
            keep='last'
        )
        checkpoint_df.to_csv('data/policy_impacts_dynamic_checkpoint.csv', index=False)
        print(f"  ✓ Checkpoint saved ({len(checkpoint_df)} unique records)")
        
    except Exception as e:
        print(f"  ✗ ERROR processing {reform_id}: {type(e).__name__}: {e}")
        print(f"  Continuing with next reform...")
        import traceback
        traceback.print_exc()

# Convert to DataFrame and deduplicate final results
dynamic_df = pd.DataFrame(dynamic_results)
if len(dynamic_df) > 0:
    dynamic_df = dynamic_df.drop_duplicates(
        subset=['reform_id', 'year', 'scoring_type'], 
        keep='last'
    )

print("\n" + "="*80)
print("DYNAMIC IMPACTS SUMMARY")
print("="*80)

if len(dynamic_df) > 0:
    # Show 2026 impacts
    dynamic_2026 = dynamic_df[dynamic_df['year'] == 2026].copy()
    dynamic_2026['revenue_impact_billions'] = dynamic_2026['revenue_impact'] / 1e9
    
    print(f"\n2026 Dynamic Impacts (Billions):")
    for _, row in dynamic_2026.iterrows():
        print(f"  {row['reform_id']}: ${row['revenue_impact_billions']:,.1f}B")
    
    print(f"\nTotal records: {len(dynamic_df)}")
else:
    print("⚠ No dynamic results calculated")


## Dynamic Impact with Age Multipliers (NEW)

Calculate dynamic budgetary impacts with the **new age-heterogeneous labor supply elasticities** from PR #6681.
This version includes 2x multipliers for individuals 65+ based on French (2005) and CBO research.

**Comparison:**
- **Dynamic (old)**: Same elasticity for all ages
- **Dynamic_multiplier (new)**: 2x elasticity for 65+ population

Since Social Security reforms primarily affect 65+ individuals, the age multipliers should produce larger behavioral effects.

**Note:** You must run the "Compute Baselines" cell above before running this cell.

In [ ]:
# Storage for dynamic_multiplier results
dynamic_multiplier_results = []

# Create data directory if it doesn't exist
import os
os.makedirs('data', exist_ok=True)

# Load existing checkpoint if it exists to avoid duplicates
checkpoint_file = 'data/policy_impacts_dynamic_multiplier_checkpoint.csv'
if os.path.exists(checkpoint_file):
    existing_df = pd.read_csv(checkpoint_file)
    dynamic_multiplier_results = existing_df.to_dict('records')
    print(f"📂 Loaded {len(dynamic_multiplier_results)} existing dynamic_multiplier results from checkpoint")
    print(f"   Reforms already computed: {sorted(existing_df['reform_id'].unique())}")
    print(f"   Years covered: {sorted(existing_df['year'].unique())}")
    print()

# Only calculate dynamic for 2026 (single year)
DYNAMIC_YEARS = [2026]

print("\n" + "="*80)
print("DYNAMIC IMPACT CALCULATIONS WITH AGE MULTIPLIERS")
print("="*80)
print(f"Analyzing {len(REFORMS)} reforms for year {DYNAMIC_YEARS[0]} only = {len(REFORMS)} calculations\n")
print("Note: This version includes 2x age multipliers for 65+ population (PR #6681)")
print("      Expect larger behavioral effects than uniform elasticity model.\n")

for reform_id, reform_config in tqdm(REFORMS.items(), desc="Reforms"):
    reform_name = reform_config['name']
    reform_func = reform_config['func']
    
    print(f"\nProcessing {reform_id}: {reform_name}")
    
    try:
        # Get the reform dict and combine with labor elasticities (including age multipliers)
        reform_dict = get_reform_dict(reform_func)
        dynamic_reform = create_dynamic_reform(reform_dict, CBO_LABOR_PARAMS)
        
        for year in DYNAMIC_YEARS:
            print(f"  Calculating {year}...", end=' ')
            
            impact = calculate_revenue_impact(dynamic_reform, year, baselines[year])
            
            dynamic_multiplier_results.append({
                'reform_id': reform_id,
                'reform_name': reform_name,
                'year': year,
                'revenue_impact': impact,
                'scoring_type': 'dynamic_multiplier'
            })
            
            print(f"${impact/1e9:.2f}B")
        
        # Deduplicate before saving checkpoint (keep most recent calculation)
        checkpoint_df = pd.DataFrame(dynamic_multiplier_results)
        checkpoint_df = checkpoint_df.drop_duplicates(
            subset=['reform_id', 'year', 'scoring_type'], 
            keep='last'
        )
        checkpoint_df.to_csv('data/policy_impacts_dynamic_multiplier_checkpoint.csv', index=False)
        print(f"  ✓ Checkpoint saved ({len(checkpoint_df)} unique records)")
        
    except Exception as e:
        print(f"  ✗ ERROR processing {reform_id}: {type(e).__name__}: {e}")
        print(f"  Continuing with next reform...")
        import traceback
        traceback.print_exc()

# Convert to DataFrame and deduplicate final results
dynamic_multiplier_df = pd.DataFrame(dynamic_multiplier_results)
if len(dynamic_multiplier_df) > 0:
    dynamic_multiplier_df = dynamic_multiplier_df.drop_duplicates(
        subset=['reform_id', 'year', 'scoring_type'], 
        keep='last'
    )

print("\n" + "="*80)
print("DYNAMIC WITH MULTIPLIERS IMPACTS SUMMARY")
print("="*80)

if len(dynamic_multiplier_df) > 0:
    # Show 2026 impacts
    dynamic_mult_2026 = dynamic_multiplier_df[dynamic_multiplier_df['year'] == 2026].copy()
    dynamic_mult_2026['revenue_impact_billions'] = dynamic_mult_2026['revenue_impact'] / 1e9
    
    print(f"\n2026 Dynamic (with Age Multipliers) Impacts (Billions):")
    for _, row in dynamic_mult_2026.iterrows():
        print(f"  {row['reform_id']}: ${row['revenue_impact_billions']:,.1f}B")
    
    print(f"\nTotal records: {len(dynamic_multiplier_df)}")
else:
    print("⚠ No dynamic_multiplier results calculated")

## Comparison: Dynamic vs Static

Calculate the difference between dynamic and static scores to show the behavioral feedback effects.

In [12]:
# Merge static and dynamic results
print("\n" + "="*80)
print("BEHAVIORAL FEEDBACK EFFECTS (2026 ONLY)")
print("="*80)

# Load static data if it wasn't computed in this run
if 'static_df' not in locals() or len(static_df) == 0:
    static_checkpoint = '../data/policy_impacts_static_checkpoint.csv'
    if os.path.exists(static_checkpoint):
        static_df = pd.read_csv(static_checkpoint)
        print(f"📂 Loaded {len(static_df)} static results from checkpoint for comparison\n")
    else:
        static_df = pd.DataFrame()
        
if 'dynamic_df' not in locals():
    dynamic_df = pd.DataFrame()

if len(static_df) == 0:
    print("⚠ No static results to compare")
elif len(dynamic_df) == 0:
    print("⚠ No dynamic results to compare")
else:
    # Filter static to 2026 only for comparison
    static_2026 = static_df[static_df['year'] == 2026].copy()
    
    comparison_df = pd.merge(
        static_2026[['reform_id', 'reform_name', 'year', 'revenue_impact']].rename(columns={'revenue_impact': 'static_impact'}),
        dynamic_df[['reform_id', 'reform_name', 'year', 'revenue_impact']].rename(columns={'revenue_impact': 'dynamic_impact'}),
        on=['reform_id', 'reform_name', 'year'],
        how='outer'  # Use outer join to keep all results even if one side is missing
    )

    # Calculate difference (dynamic - static)
    # Positive difference = behavioral responses increase revenue relative to static
    # Negative difference = behavioral responses decrease revenue relative to static
    comparison_df['dynamic_feedback'] = comparison_df['dynamic_impact'] - comparison_df['static_impact']

    # Calculate percentage difference
    comparison_df['feedback_pct'] = (comparison_df['dynamic_feedback'] / comparison_df['static_impact'].abs()) * 100

    # Show 2026 feedback effects (only for reforms with both static and dynamic)
    complete_reforms = comparison_df.dropna(subset=['static_impact', 'dynamic_impact'])
    
    if len(complete_reforms) > 0:
        feedback_summary = complete_reforms.groupby(['reform_id', 'reform_name']).agg({
            'static_impact': 'sum',
            'dynamic_impact': 'sum',
            'dynamic_feedback': 'sum'
        }) / 1e9

        feedback_summary['feedback_pct'] = (feedback_summary['dynamic_feedback'] / feedback_summary['static_impact'].abs()) * 100

        print("\n2026 Comparison (Static vs Dynamic, Billions):")
        print("-" * 80)
        for (reform_id, reform_name), row in feedback_summary.iterrows():
            print(f"\n{reform_id}: {reform_name}")
            print(f"  Static impact:     ${row['static_impact']:>8,.1f}B")
            print(f"  Dynamic impact:    ${row['dynamic_impact']:>8,.1f}B")
            print(f"  Feedback effect:   ${row['dynamic_feedback']:>8,.1f}B ({row['feedback_pct']:+.1f}%)")

        print(f"\n✓ Comparison complete for {len(feedback_summary)} reforms")
    else:
        print("\n⚠ No reforms have both static and dynamic results to compare")
        comparison_df = pd.DataFrame()  # Empty dataframe

    comparison_df.head()


BEHAVIORAL FEEDBACK EFFECTS (2026 ONLY)
📂 Loaded 80 static results from checkpoint for comparison


2026 Comparison (Static vs Dynamic, Billions):
--------------------------------------------------------------------------------

option1: Full Repeal of Social Security Benefits Taxation
  Static impact:     $   -85.4B
  Dynamic impact:    $   -85.0B
  Feedback effect:   $     0.4B (+0.4%)

option2: Taxation of 85% of Social Security Benefits
  Static impact:     $    24.3B
  Dynamic impact:    $    24.5B
  Feedback effect:   $     0.2B (+1.0%)

option3: 85% Taxation with Permanent Senior Deduction Extension
  Static impact:     $    24.3B
  Dynamic impact:    $    24.5B
  Feedback effect:   $     0.2B (+1.0%)

option4: Social Security Tax Credit System ($500)
  Static impact:     $    29.9B
  Dynamic impact:    $    30.0B
  Feedback effect:   $     0.2B (+0.5%)

option5: Roth-Style Swap
  Static impact:     $    58.1B
  Dynamic impact:    $    42.0B
  Feedback effect:   $   -16.1B (-27

## 3 Comparison: Static vs Dynamic vs Dynamic with Multipliers

Compare all three scoring methodologies to understand the impact of behavioral responses and age heterogeneity:

1. **Static**: No behavioral response (baseline)
2. **Dynamic**: Uniform labor supply elasticities (same for all ages)
3. **Dynamic with Multipliers**: Age-heterogeneous elasticities (2x for 65+)

This comparison shows:
- Static → Dynamic: Effect of adding behavioral responses
- Dynamic → Dynamic_Multiplier: Effect of adding age heterogeneity
- Static → Dynamic_Multiplier: Total effect of realistic behavioral modeling

In [ ]:
# Three-way comparison: Static vs Dynamic vs Dynamic_Multiplier
print("\n" + "="*80)
print("THREE-WAY COMPARISON: STATIC VS DYNAMIC VS DYNAMIC WITH MULTIPLIERS (2026)")
print("="*80)

# Load data from variables or checkpoints
if 'static_df' not in locals() or len(static_df) == 0:
    static_checkpoint = 'data/policy_impacts_static_checkpoint.csv'
    if os.path.exists(static_checkpoint):
        static_df = pd.read_csv(static_checkpoint)
        print(f"📂 Loaded {len(static_df)} static results from checkpoint")
    else:
        static_df = pd.DataFrame()
        
if 'dynamic_df' not in locals() or len(dynamic_df) == 0:
    dynamic_checkpoint = 'data/policy_impacts_dynamic_checkpoint.csv'
    if os.path.exists(dynamic_checkpoint):
        dynamic_df = pd.read_csv(dynamic_checkpoint)
        print(f"📂 Loaded {len(dynamic_df)} dynamic results from checkpoint")
    else:
        dynamic_df = pd.DataFrame()

if 'dynamic_multiplier_df' not in locals() or len(dynamic_multiplier_df) == 0:
    dynamic_mult_checkpoint = 'data/policy_impacts_dynamic_multiplier_checkpoint.csv'
    if os.path.exists(dynamic_mult_checkpoint):
        dynamic_multiplier_df = pd.read_csv(dynamic_mult_checkpoint)
        print(f"📂 Loaded {len(dynamic_multiplier_df)} dynamic_multiplier results from checkpoint")
    else:
        dynamic_multiplier_df = pd.DataFrame()

print()

# Check if we have all three datasets
if len(static_df) == 0:
    print("⚠ No static results to compare")
    comparison_3way_export = pd.DataFrame()
elif len(dynamic_df) == 0:
    print("⚠ No dynamic results to compare")
    comparison_3way_export = pd.DataFrame()
elif len(dynamic_multiplier_df) == 0:
    print("⚠ No dynamic_multiplier results to compare")
    comparison_3way_export = pd.DataFrame()
else:
    # Filter all to 2026 only
    static_2026 = static_df[static_df['year'] == 2026].copy()
    dynamic_2026 = dynamic_df[dynamic_df['year'] == 2026].copy()
    dynamic_mult_2026 = dynamic_multiplier_df[dynamic_multiplier_df['year'] == 2026].copy()
    
    # Merge all three
    comparison_3way = static_2026[['reform_id', 'reform_name', 'year', 'revenue_impact']].rename(
        columns={'revenue_impact': 'static_impact'}
    )
    
    comparison_3way = comparison_3way.merge(
        dynamic_2026[['reform_id', 'year', 'revenue_impact']].rename(columns={'revenue_impact': 'dynamic_impact'}),
        on=['reform_id', 'year'],
        how='outer'
    )
    
    comparison_3way = comparison_3way.merge(
        dynamic_mult_2026[['reform_id', 'year', 'revenue_impact']].rename(columns={'revenue_impact': 'dynamic_multiplier_impact'}),
        on=['reform_id', 'year'],
        how='outer'
    )
    
    # Calculate differences
    # Behavioral effect (static → dynamic)
    comparison_3way['behavioral_effect'] = comparison_3way['dynamic_impact'] - comparison_3way['static_impact']
    comparison_3way['behavioral_pct'] = (comparison_3way['behavioral_effect'] / comparison_3way['static_impact'].abs()) * 100
    
    # Age heterogeneity effect (dynamic → dynamic_multiplier)
    comparison_3way['age_effect'] = comparison_3way['dynamic_multiplier_impact'] - comparison_3way['dynamic_impact']
    comparison_3way['age_effect_pct'] = (comparison_3way['age_effect'] / comparison_3way['dynamic_impact'].abs()) * 100
    
    # Total effect (static → dynamic_multiplier)
    comparison_3way['total_effect'] = comparison_3way['dynamic_multiplier_impact'] - comparison_3way['static_impact']
    comparison_3way['total_pct'] = (comparison_3way['total_effect'] / comparison_3way['static_impact'].abs()) * 100
    
    # Filter to complete reforms only (have all three estimates)
    complete_3way = comparison_3way.dropna(subset=['static_impact', 'dynamic_impact', 'dynamic_multiplier_impact'])
    
    if len(complete_3way) > 0:
        # Convert to billions for display
        summary_3way = complete_3way.groupby(['reform_id', 'reform_name']).agg({
            'static_impact': 'sum',
            'dynamic_impact': 'sum',
            'dynamic_multiplier_impact': 'sum',
            'behavioral_effect': 'sum',
            'age_effect': 'sum',
            'total_effect': 'sum'
        }) / 1e9
        
        # Recalculate percentages for aggregated data
        summary_3way['behavioral_pct'] = (summary_3way['behavioral_effect'] / summary_3way['static_impact'].abs()) * 100
        summary_3way['age_effect_pct'] = (summary_3way['age_effect'] / summary_3way['dynamic_impact'].abs()) * 100
        summary_3way['total_pct'] = (summary_3way['total_effect'] / summary_3way['static_impact'].abs()) * 100
        
        print("\n2026 Three-Way Comparison (Billions):")
        print("="*80)
        
        for (reform_id, reform_name), row in summary_3way.iterrows():
            print(f"\n{reform_id}: {reform_name}")
            print(f"  Static impact:              ${row['static_impact']:>8,.1f}B")
            print(f"  Dynamic impact:             ${row['dynamic_impact']:>8,.1f}B")
            print(f"  Dynamic w/ multipliers:     ${row['dynamic_multiplier_impact']:>8,.1f}B")
            print(f"  ─" * 40)
            print(f"  Behavioral effect:          ${row['behavioral_effect']:>8,.1f}B ({row['behavioral_pct']:+.1f}%)")
            print(f"  Age heterogeneity effect:   ${row['age_effect']:>8,.1f}B ({row['age_effect_pct']:+.1f}%)")
            print(f"  Total dynamic effect:       ${row['total_effect']:>8,.1f}B ({row['total_pct']:+.1f}%)")
        
        print(f"\n✓ Three-way comparison complete for {len(summary_3way)} reforms")
        
        # Store for export
        comparison_3way_export = comparison_3way.copy()
        
        # Display sample
        display(comparison_3way.head())
    else:
        print("\n⚠ No reforms have all three estimates (static, dynamic, dynamic_multiplier)")
        comparison_3way_export = pd.DataFrame()

## Export Results

Save all results to CSV files for use in other analyses and visualizations.

In [7]:
import os

# Create data directory if it doesn't exist
os.makedirs('../data', exist_ok=True)

# Load data from variables or checkpoints
if 'static_df' not in locals():
    static_checkpoint = '../data/policy_impacts_static_checkpoint.csv'
    if os.path.exists(static_checkpoint):
        static_df = pd.read_csv(static_checkpoint)
    else:
        static_df = pd.DataFrame()
        
if 'dynamic_df' not in locals():
    dynamic_checkpoint = '../data/policy_impacts_dynamic_checkpoint.csv'
    if os.path.exists(dynamic_checkpoint):
        dynamic_df = pd.read_csv(dynamic_checkpoint)
    else:
        dynamic_df = pd.DataFrame()

if 'dynamic_multiplier_df' not in locals():
    dynamic_mult_checkpoint = '../data/policy_impacts_dynamic_multiplier_checkpoint.csv'
    if os.path.exists(dynamic_mult_checkpoint):
        dynamic_multiplier_df = pd.read_csv(dynamic_mult_checkpoint)
    else:
        dynamic_multiplier_df = pd.DataFrame()

print("\n" + "="*80)
print("EXPORTING RESULTS")
print("="*80)

# Export static impacts (10 years: 2026-2035)
if len(static_df) > 0:
    static_df.to_csv('../data/policy_impacts_static.csv', index=False)
    print("✓ Exported static impacts to: data/policy_impacts_static.csv")
    print(f"  ({len(static_df)} records - 10 years: 2026-2035)")
    
    # Also export a legacy version for backward compatibility
    static_df.to_csv('../data/policy_impacts.csv', index=False)
    print("✓ Exported static impacts (legacy) to: data/policy_impacts.csv")
else:
    print("⚠ No static impacts to export")

# Export dynamic impacts (old - without age multipliers)
if len(dynamic_df) > 0:
    dynamic_df.to_csv('../data/policy_impacts_dynamic.csv', index=False)
    print("✓ Exported dynamic impacts to: data/policy_impacts_dynamic.csv")
    print(f"  ({len(dynamic_df)} records - 2026 only, uniform elasticities)")
else:
    print("⚠ No dynamic impacts to export")

# Export dynamic_multiplier impacts (new - with age multipliers)
if len(dynamic_multiplier_df) > 0:
    dynamic_multiplier_df.to_csv('../data/policy_impacts_dynamic_multiplier.csv', index=False)
    print("✓ Exported dynamic_multiplier impacts to: data/policy_impacts_dynamic_multiplier.csv")
    print(f"  ({len(dynamic_multiplier_df)} records - 2026 only, age-heterogeneous elasticities)")
else:
    print("⚠ No dynamic_multiplier impacts to export")

# Export old two-way comparison data (static vs dynamic)
if 'comparison_df' in locals() and len(comparison_df) > 0:
    comparison_df.to_csv('../data/policy_impacts_comparison.csv', index=False)
    print("✓ Exported 2-way comparison data to: data/policy_impacts_comparison.csv")
    print(f"  ({len(comparison_df)} records - static vs dynamic)")
    
    # Export summary statistics
    if 'feedback_summary' in locals() and len(feedback_summary) > 0:
        summary_stats = pd.DataFrame({
            'reform_id': feedback_summary.index.get_level_values(0),
            'reform_name': feedback_summary.index.get_level_values(1),
            'static_2026_billions': feedback_summary['static_impact'].values,
            'dynamic_2026_billions': feedback_summary['dynamic_impact'].values,
            'feedback_billions': feedback_summary['dynamic_feedback'].values,
            'feedback_percent': feedback_summary['feedback_pct'].values
        })
        summary_stats.to_csv('../data/policy_impacts_summary.csv', index=False)
        print("✓ Exported 2-way summary statistics to: data/policy_impacts_summary.csv")

# Export full comparison data (static vs dynamic vs dynamic_multiplier)
if 'comparison_3way_export' in locals() and len(comparison_3way_export) > 0:
    comparison_3way_export.to_csv('../data/policy_impacts_comparison_full.csv', index=False)
    print("✓ Exported full comparison data to: data/policy_impacts_comparison_full.csv")
    print(f"  ({len(comparison_3way_export)} records - static vs dynamic vs dynamic_multiplier)")
    
    # Export full summary statistics
    if 'summary_3way' in locals() and len(summary_3way) > 0:
        summary_3way_stats = pd.DataFrame({
            'reform_id': summary_3way.index.get_level_values(0),
            'reform_name': summary_3way.index.get_level_values(1),
            'static_2026_billions': summary_3way['static_impact'].values,
            'dynamic_2026_billions': summary_3way['dynamic_impact'].values,
            'dynamic_multiplier_2026_billions': summary_3way['dynamic_multiplier_impact'].values,
            'behavioral_effect_billions': summary_3way['behavioral_effect'].values,
            'behavioral_effect_percent': summary_3way['behavioral_pct'].values,
            'age_effect_billions': summary_3way['age_effect'].values,
            'age_effect_percent': summary_3way['age_effect_pct'].values,
            'total_effect_billions': summary_3way['total_effect'].values,
            'total_effect_percent': summary_3way['total_pct'].values
        })
        summary_3way_stats.to_csv('../data/policy_impacts_summary_full.csv', index=False)
        print("✓ Exported full summary statistics to: data/policy_impacts_summary_full.csv")
        print(f"  ({len(summary_3way_stats)} reforms)")

# Clean up checkpoint files if everything completed successfully
checkpoints_to_remove = []
if len(static_df) > 0:
    checkpoints_to_remove.append('../data/policy_impacts_static_checkpoint.csv')
if len(dynamic_df) > 0:
    checkpoints_to_remove.append('../data/policy_impacts_dynamic_checkpoint.csv')
if len(dynamic_multiplier_df) > 0:
    checkpoints_to_remove.append('../data/policy_impacts_dynamic_multiplier_checkpoint.csv')

if len(checkpoints_to_remove) > 0:
    print()
    for checkpoint in checkpoints_to_remove:
        if os.path.exists(checkpoint):
            os.remove(checkpoint)
            print(f"✓ Removed checkpoint: {os.path.basename(checkpoint)}")

print("\n" + "="*80)
print("EXPORT SUMMARY")
print("="*80)
print(f"Static reforms processed: {static_df['reform_id'].nunique() if len(static_df) > 0 else 0} (2026-2035)")
print(f"Dynamic reforms processed: {dynamic_df['reform_id'].nunique() if len(dynamic_df) > 0 else 0} (2026 only, uniform)")
print(f"Dynamic_multiplier reforms processed: {dynamic_multiplier_df['reform_id'].nunique() if len(dynamic_multiplier_df) > 0 else 0} (2026 only, age-heterogeneous)")
print(f"\nAll exports complete!")


EXPORTING RESULTS
⚠ No static impacts to export
⚠ No dynamic impacts to export
✓ Exported dynamic_multiplier impacts to: data/policy_impacts_dynamic_multiplier.csv
  (8 records - 2026 only, age-heterogeneous elasticities)

✓ Removed checkpoint: policy_impacts_dynamic_multiplier_checkpoint.csv

EXPORT SUMMARY
Static reforms processed: 0 (2026-2035)
Dynamic reforms processed: 0 (2026 only, uniform)
Dynamic_multiplier reforms processed: 8 (2026 only, age-heterogeneous)

All exports complete!
